# PyTeNet Basics

This tutorial shows how to construct a matrix product state (MPS) with random entries, a Hamiltonian in matrix product operator (MPO) form and compute the corresponding average energy.

In [1]:
# import NumPy and the PyTeNet package
import numpy as np
import pytenet as ptn

In [2]:
# seed random number generator for reproducibility
np.random.seed(42)

## Matrix product states

Create a matrix product state with random tensor entries:

In [3]:
# physical dimension (of each lattice site);
# this could be spin 1, 0, -1 for example, or 0, 1, 2 particles per lattice site
d = 3

# virtual bond dimensions (note the leading and trailing 1)
D = [1, 4, 15, 13, 7, 1]

# set all physical and virtual bond quantum numbers to zero
# (effectively disabling quantum numbers)
qd = np.zeros(d, dtype=int)
qD = [np.zeros(Di, dtype=int) for Di in D]

# now create the matrix product state
mps = ptn.MPS(qd, qD, fill='random')

The actual tensors of the MPS are stored in the `mps.A` instance variable (list of NumPy arrays).

In [4]:
type(mps.A)

list

In [5]:
type(mps.A[0])

numpy.ndarray

In [6]:
# the first dimension is the physical dimension,
# and the second and third dimension the left and right virtual bond dimensions
mps.A[0].shape

(3, 1, 4)

In [7]:
# show random entries of leftmost tensor as illustration
mps.A[0]

array([[[ 0.14338902+0.06984849j, -0.03991347-0.55231643j,
          0.18697158-0.49794089j,  0.43966085-0.16231843j]],

       [[-0.06759426-0.29237916j, -0.06758952+0.09071539j,
          0.45587947-0.26212397j,  0.22153932-0.40769696j]],

       [[-0.13552558+0.42309636j,  0.15662359-0.065176j  ,
         -0.13377716+0.01949371j, -0.1344446 -0.41128937j]]])

In [8]:
# the number of lattice sites is (by definition) the length of mps.A;
# by construction same as length of virtual bond dimension list - 1
L = mps.nsites
(L, len(mps.A), len(D)-1)

(5, 5, 5)

Here are our virtual bond dimensions (same as `D` above):

In [9]:
mps.bond_dims

[1, 4, 15, 13, 7, 1]

The `as_vector()` function computes the vector representation of a matrix product state:

In [10]:
x = mps.as_vector()
# the length of x is equal to the physical Hilbert space dimension,
# i.e., d^L with L the number of lattice sites:
(x.shape, d**L)

((243,), 243)

In [11]:
# in general, the MPS is not normalized
np.linalg.norm(x)

0.06088431822262377

Call `mps.orthonormalize()` to left- or right-orthonormalize the `mps.A` tensors (via QR decompositions). This function returns the hitherto norm.

In [12]:
# returns previous norm
mps.orthonormalize(mode='left')

0.06088431822262377

In [13]:
# now the norm is 1, as expected
x = mps.as_vector()
np.linalg.norm(x)

1.0000000000000002

## Hamiltonian as matrix product operator

PyTeNet supports some common quantum Hamiltonians in MPO form, see `hamiltonian.py`. The construction method there is very general and straightforward to adapt (see the `hamiltonian_mpo.ipynb` tutorial). Here we use the Bose-Hubbard model with the following Hamiltonian as illustration.

$$
H = -t \sum_{j=1}^{L-1} \left(b^{\dagger}_j b_{j+1} + \text{h.c.}\right) + \tfrac{1}{2} U \sum_{j=1}^L \hat{n}_j \left(\hat{n}_j - 1\right) - \mu \sum_{j=1}^L \hat{n}_j
$$

In [14]:
# Hamiltonian parameters
t  =  1.0
U  =  4.0
mu = -0.5
# construct MPO (allowed local occupancies are 0, 1, ..., d - 1)
BH = ptn.bose_hubbard_MPO(d, L, t, U, mu)

As for matrix product states, the actual tensors of the MPO are stored in the `BH.A` instance variable (list of NumPy arrays):

In [15]:
type(BH.A)

list

In [16]:
type(BH.A[0])

numpy.ndarray

In [17]:
# the first and second dimensions are the physical dimensions,
# and the third and fourth dimension the left and right virtual bond dimensions
(BH.A[0].shape, BH.A[1].shape, BH.A[2].shape)

((3, 3, 1, 4), (3, 3, 4, 4), (3, 3, 4, 4))

In [18]:
# tensors are quite sparse
BH.A[0]

array([[[[ 0.        +0.j,  0.        +0.j,  0.        +0.j,
           1.        +0.j]],

        [[ 0.        +0.j,  0.        +0.j,  1.        +0.j,
           0.        +0.j]],

        [[ 0.        +0.j,  0.        +0.j,  0.        +0.j,
           0.        +0.j]]],


       [[[ 0.        +0.j, -1.        +0.j,  0.        +0.j,
           0.        +0.j]],

        [[ 0.5       +0.j,  0.        +0.j,  0.        +0.j,
           1.        +0.j]],

        [[ 0.        +0.j,  0.        +0.j,  1.41421356+0.j,
           0.        +0.j]]],


       [[[ 0.        +0.j,  0.        +0.j,  0.        +0.j,
           0.        +0.j]],

        [[ 0.        +0.j, -1.41421356+0.j,  0.        +0.j,
           0.        +0.j]],

        [[ 5.        +0.j,  0.        +0.j,  0.        +0.j,
           1.        +0.j]]]])

The `as_matrix()` function computes the matrix representation of a matrix product operator:

In [19]:
BH_mat = BH.as_matrix()

In [20]:
type(BH_mat)

numpy.ndarray

In [21]:
BH_mat.shape

(243, 243)

## Expectation values

Now let's compute the average energy (expectation value) $\langle\psi \vert H \vert \psi\rangle$ via the MPS formalism:

In [22]:
ptn.operator_average(mps, BH)

(9.14815014704742+0j)

In [23]:
# should agree with matrix-vector representation
np.vdot(x, np.dot(BH_mat, x))

(9.148150147047424+0j)